# Visualization Examples with PyMevol Plus

This notebook demonstrates how to create compelling visualizations of API evolution data using PyMevol Plus with various plotting libraries.

## What You'll Learn:
- 📊 Creating timeline charts of API evolution
- 🏗️ Visualizing API type distributions
- 🔄 Plotting change patterns and trends
- 📈 Building interactive dashboards
- 🎨 Customizing visualizations for reports

## Libraries Used:
- **Plotly**: Interactive charts and graphs
- **Matplotlib**: Static publication-quality plots
- **Seaborn**: Statistical visualizations
- **Pandas**: Data manipulation and analysis

## 1. Setup and Sample Data

Let's start by importing libraries and creating sample data for visualization.

In [ ]:
# Core imports
import json
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from collections import defaultdict, Counter

# PyMevol Plus imports
from pymevol.models import APIElement, APIType, VersionInfo, APIChange, AnalysisResult

# Visualization libraries
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.offline as pyo

# Enable offline plotting
pyo.init_notebook_mode(connected=True)

# Optional libraries (install if needed)
try:
    import matplotlib.pyplot as plt
    import seaborn as sns
    HAS_MPL = True
    # Set style
    plt.style.use('default')
    sns.set_palette("husl")
except ImportError:
    HAS_MPL = False
    print("💡 Matplotlib/Seaborn not available. Install with: pip install matplotlib seaborn")

print("🎨 Visualization libraries loaded!")
print(f"📊 Plotly available: ✅")
print(f"📈 Matplotlib/Seaborn available: {'✅' if HAS_MPL else '❌'}")

In [ ]:
# Create rich sample data for visualization
def create_sample_evolution_data():
    """Create a realistic dataset showing API evolution over time."""
    
    # Define versions with realistic dates
    base_date = datetime(2020, 1, 1)
    versions = [
        VersionInfo("1.0.0", (base_date + timedelta(days=0)).strftime("%Y-%m-%d")),
        VersionInfo("1.1.0", (base_date + timedelta(days=90)).strftime("%Y-%m-%d")),
        VersionInfo("1.2.0", (base_date + timedelta(days=180)).strftime("%Y-%m-%d")),
        VersionInfo("2.0.0", (base_date + timedelta(days=365)).strftime("%Y-%m-%d")),
        VersionInfo("2.1.0", (base_date + timedelta(days=450)).strftime("%Y-%m-%d")),
        VersionInfo("2.2.0", (base_date + timedelta(days=540)).strftime("%Y-%m-%d")),
        VersionInfo("3.0.0", (base_date + timedelta(days=730)).strftime("%Y-%m-%d")),
    ]
    
    # Create API elements for each version (simulating evolution)
    api_elements = {}
    changes = []
    
    # Base APIs (v1.0.0)
    v1_apis = [
        APIElement("connect", APIType.FUNCTION, "mylib.core"),
        APIElement("disconnect", APIType.FUNCTION, "mylib.core"),
        APIElement("Client", APIType.CLASS, "mylib.client"),
        APIElement("Logger", APIType.CLASS, "mylib.utils"),
        APIElement("VERSION", APIType.CONSTANT, "mylib"),
    ]
    api_elements["1.0.0"] = v1_apis
    
    # v1.1.0 - Add some APIs
    v1_1_apis = v1_apis.copy() + [
        APIElement("reconnect", APIType.FUNCTION, "mylib.core"),
        APIElement("get_status", APIType.METHOD, "mylib.client.Client"),
        APIElement("Config", APIType.CLASS, "mylib.config"),
    ]
    api_elements["1.1.0"] = v1_1_apis
    changes.extend([
        APIChange("reconnect", "1.1.0", "added", "Added automatic reconnection function"),
        APIChange("get_status", "1.1.0", "added", "Added status checking method to Client"),
        APIChange("Config", "1.1.0", "added", "Added configuration management class"),
    ])
    
    # v1.2.0 - More additions
    v1_2_apis = v1_1_apis.copy() + [
        APIElement("async_connect", APIType.FUNCTION, "mylib.async_core"),
        APIElement("AsyncClient", APIType.CLASS, "mylib.async_client"),
        APIElement("validate_config", APIType.FUNCTION, "mylib.config"),
    ]
    api_elements["1.2.0"] = v1_2_apis
    changes.extend([
        APIChange("async_connect", "1.2.0", "added", "Added async support"),
        APIChange("AsyncClient", "1.2.0", "added", "Added async client class"),
        APIChange("validate_config", "1.2.0", "added", "Added config validation"),
    ])
    
    # v2.0.0 - Major version with breaking changes
    v2_0_apis = [
        # Keep most APIs but modify some
        APIElement("connect", APIType.FUNCTION, "mylib.core"),  # Modified signature
        APIElement("disconnect", APIType.FUNCTION, "mylib.core"),
        APIElement("Client", APIType.CLASS, "mylib.client"),  # Modified
        APIElement("Logger", APIType.CLASS, "mylib.logging"),  # Moved module
        APIElement("VERSION", APIType.CONSTANT, "mylib"),
        APIElement("reconnect", APIType.FUNCTION, "mylib.core"),
        APIElement("get_status", APIType.METHOD, "mylib.client.Client"),
        # Config class removed, replaced with new one
        APIElement("Settings", APIType.CLASS, "mylib.config"),  # Replaced Config
        APIElement("async_connect", APIType.FUNCTION, "mylib.async_core"),
        APIElement("AsyncClient", APIType.CLASS, "mylib.async_client"),
        # validate_config removed
        # New APIs
        APIElement("Connection", APIType.CLASS, "mylib.connection"),
        APIElement("authenticate", APIType.FUNCTION, "mylib.auth"),
    ]
    api_elements["2.0.0"] = v2_0_apis
    changes.extend([
        APIChange("connect", "2.0.0", "modified", "Changed signature - now requires auth", is_backwards_compatible=False),
        APIChange("Client", "2.0.0", "modified", "Restructured Client class", is_backwards_compatible=False),
        APIChange("Logger", "2.0.0", "modified", "Moved to mylib.logging module", is_backwards_compatible=False),
        APIChange("Config", "2.0.0", "removed", "Replaced with Settings class", is_backwards_compatible=False),
        APIChange("validate_config", "2.0.0", "removed", "Validation now built into Settings", is_backwards_compatible=False),
        APIChange("Settings", "2.0.0", "added", "New configuration management"),
        APIChange("Connection", "2.0.0", "added", "New connection abstraction"),
        APIChange("authenticate", "2.0.0", "added", "New authentication system"),
    ])
    
    # Continue evolution for remaining versions...
    # (Simplified for brevity, but following similar patterns)
    
    # v2.1.0 - Minor additions
    v2_1_apis = v2_0_apis.copy() + [
        APIElement("bulk_connect", APIType.FUNCTION, "mylib.core"),
        APIElement("ConnectionPool", APIType.CLASS, "mylib.connection"),
    ]
    api_elements["2.1.0"] = v2_1_apis
    changes.extend([
        APIChange("bulk_connect", "2.1.0", "added", "Added bulk connection support"),
        APIChange("ConnectionPool", "2.1.0", "added", "Added connection pooling"),
    ])
    
    # v2.2.0 - More features
    v2_2_apis = v2_1_apis.copy() + [
        APIElement("monitor", APIType.FUNCTION, "mylib.monitoring"),
        APIElement("Metrics", APIType.CLASS, "mylib.monitoring"),
        APIElement("export_metrics", APIType.FUNCTION, "mylib.monitoring"),
    ]
    api_elements["2.2.0"] = v2_2_apis
    changes.extend([
        APIChange("monitor", "2.2.0", "added", "Added monitoring capabilities"),
        APIChange("Metrics", "2.2.0", "added", "Added metrics collection"),
        APIChange("export_metrics", "2.2.0", "added", "Added metrics export"),
    ])
    
    # v3.0.0 - Another major version
    v3_0_apis = [
        # Streamlined API, some removals
        APIElement("connect", APIType.FUNCTION, "mylib.core"),
        APIElement("disconnect", APIType.FUNCTION, "mylib.core"),
        APIElement("Client", APIType.CLASS, "mylib.client"),
        APIElement("Logger", APIType.CLASS, "mylib.logging"),
        APIElement("VERSION", APIType.CONSTANT, "mylib"),
        # reconnect removed in favor of auto-reconnection
        APIElement("get_status", APIType.METHOD, "mylib.client.Client"),
        APIElement("Settings", APIType.CLASS, "mylib.config"),
        APIElement("Connection", APIType.CLASS, "mylib.connection"),
        APIElement("authenticate", APIType.FUNCTION, "mylib.auth"),
        APIElement("ConnectionPool", APIType.CLASS, "mylib.connection"),
        APIElement("Metrics", APIType.CLASS, "mylib.monitoring"),
        # New v3 APIs
        APIElement("Session", APIType.CLASS, "mylib.session"),
        APIElement("create_session", APIType.FUNCTION, "mylib.session"),
        APIElement("Plugin", APIType.CLASS, "mylib.plugins"),
        APIElement("load_plugin", APIType.FUNCTION, "mylib.plugins"),
    ]
    api_elements["3.0.0"] = v3_0_apis
    changes.extend([
        APIChange("async_connect", "3.0.0", "removed", "Merged into main connect function", is_backwards_compatible=False),
        APIChange("AsyncClient", "3.0.0", "removed", "Async support now built into Client", is_backwards_compatible=False),
        APIChange("reconnect", "3.0.0", "removed", "Auto-reconnection now default", is_backwards_compatible=False),
        APIChange("bulk_connect", "3.0.0", "removed", "Use ConnectionPool instead", is_backwards_compatible=False),
        APIChange("monitor", "3.0.0", "removed", "Integrated into Metrics class", is_backwards_compatible=False),
        APIChange("export_metrics", "3.0.0", "removed", "Now a method of Metrics", is_backwards_compatible=False),
        APIChange("Session", "3.0.0", "added", "New session management system"),
        APIChange("create_session", "3.0.0", "added", "Session factory function"),
        APIChange("Plugin", "3.0.0", "added", "Plugin system for extensibility"),
        APIChange("load_plugin", "3.0.0", "added", "Plugin loading mechanism"),
    ])
    
    # Create the analysis result
    result = AnalysisResult(
        package_name="mylib",
        versions=versions,
        api_elements=api_elements,
        changes=changes,
        metadata={
            "analysis_tool": "PyMevol Plus",
            "sample_data": True,
            "created_for": "visualization_demo"
        }
    )
    
    return result

# Create our sample data
demo_result = create_sample_evolution_data()
print(f"📊 Sample data created for '{demo_result.package_name}'")
print(f"📈 Versions: {len(demo_result.versions)}")
print(f"🔄 Changes: {len(demo_result.changes)}")
print(f"📋 API elements: {sum(len(apis) for apis in demo_result.api_elements.values())}")

## 2. API Evolution Timeline

The timeline view is one of the most powerful ways to visualize API evolution, showing how APIs are introduced, modified, and removed over time.

In [ ]:
# Create API evolution timeline
def create_api_timeline(result):\n    \"\"\"Create an interactive timeline showing API lifecycle.\"\"\"\n    \n    # Prepare data for timeline\n    timeline_data = []\n    colors = {\n        'added': '#2E8B57',      # Sea Green\n        'modified': '#FF8C00',   # Dark Orange  \n        'removed': '#DC143C',    # Crimson\n        'deprecated': '#B8860B'  # Dark Goldenrod\n    }\n    \n    # Convert version dates for plotting\n    version_dates = {}\n    for version in result.versions:\n        try:\n            version_dates[version.number] = pd.to_datetime(version.release_date)\n        except:\n            # Fallback if date parsing fails\n            version_dates[version.number] = pd.to_datetime('2020-01-01')\n    \n    # Process each change\n    for change in result.changes:\n        if change.version in version_dates:\n            timeline_data.append({\n                'version': change.version,\n                'date': version_dates[change.version],\n                'api_name': change.api_name,\n                'change_type': change.change_type,\n                'description': change.description or f\"{change.change_type.title()} {change.api_name}\",\n                'is_breaking': not change.is_backwards_compatible,\n                'color': colors.get(change.change_type, '#808080')\n            })\n    \n    # Create DataFrame\n    df = pd.DataFrame(timeline_data)\n    \n    if df.empty:\n        print(\"No timeline data available\")\n        return None\n    \n    # Create the timeline plot\n    fig = go.Figure()\n    \n    # Add traces for each change type\n    for change_type in df['change_type'].unique():\n        type_data = df[df['change_type'] == change_type]\n        \n        fig.add_trace(go.Scatter(\n            x=type_data['date'],\n            y=type_data['api_name'],\n            mode='markers',\n            name=change_type.title(),\n            marker=dict(\n                size=12,\n                color=colors.get(change_type, '#808080'),\n                symbol='diamond' if type_data['is_breaking'].any() else 'circle',\n                line=dict(width=2, color='white')\n            ),\n            text=type_data['description'],\n            hovertemplate=\n                \"<b>%{y}</b><br>\" +\n                \"Date: %{x}<br>\" +\n                \"Type: \" + change_type + \"<br>\" +\n                \"Description: %{text}<br>\" +\n                \"<extra></extra>\"\n        ))\n    \n    # Customize layout\n    fig.update_layout(\n        title={\n            'text': f'API Evolution Timeline - {result.package_name}',\n            'x': 0.5,\n            'font': {'size': 20}\n        },\n        xaxis_title='Release Date',\n        yaxis_title='API Elements',\n        height=max(600, len(df) * 25),  # Adjust height based on data\n        hovermode='closest',\n        legend=dict(\n            orientation=\"h\",\n            yanchor=\"bottom\",\n            y=1.02,\n            xanchor=\"right\",\n            x=1\n        ),\n        font=dict(size=12)\n    )\n    \n    # Add version lines\n    for version, date in version_dates.items():\n        fig.add_vline(\n            x=date, \n            line_dash=\"dash\", \n            line_color=\"gray\",\n            annotation_text=f\"v{version}\",\n            annotation_position=\"top\"\n        )\n    \n    return fig\n\n# Create and display the timeline\ntimeline_fig = create_api_timeline(demo_result)\nif timeline_fig:\n    timeline_fig.show()\n    print(\"📊 Interactive timeline created! Click and drag to zoom, hover for details.\")\nelse:\n    print(\"❌ Could not create timeline visualization\")"